In [2]:
import sqlite3
import folium

def create_station_map(database_name: str):
    conn = sqlite3.connect(database_name)
    cursor = conn.cursor()

    cursor.execute("SELECT station_name, latitude, longitude FROM STATIONS WHERE latitude IS NOT NULL AND longitude IS NOT NULL")
    stations = cursor.fetchall()
    conn.close()

    poland_center = [52.0, 19.0]

    mapa = folium.Map(location=poland_center, zoom_start=6)

    for name, lat, lon in stations:
        folium.Marker(
            location=[lat, lon],
            popup=name,
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(mapa)

    return mapa

create_station_map("data.db")
